## Competitor Price Watchdog

#### This script is an example of a **Workflow** (or Pipeline), not an **Agent**. 

In [1]:
import requests
from typing import TypedDict, List, Optional, Dict
from langgraph.graph import StateGraph, END

In [2]:
# The STATE

class DealState(TypedDict):
    search_query: str
    target_price: float
    
    # Internal Data
    game_id: Optional[str] # The API's specific ID for the game
    game_title: Optional[str] # The API's official title for the game
    deals: List[Dict] # Raw list of prices from all stores
    best_deal: Optional[Dict] # The best deal found

    # Output Data
    decision: str # "buy", "wait", "error"
    message: str # Final report

### The NODES - Search game, fetch price, analyzer, notifier

In [3]:
# Node 1 - Search Game

def search_game_node(state: DealState):
    """
    Step 1: Real API call to resolve the user's query into a Game ID.
    """
    query = state['search_query']
    print(f"--- 1. SEARCHING: Looking for '{query}' in public database... ---")
    
    # REAL DATA SOURCE: CheapShark API
    url = f"https://www.cheapshark.com/api/1.0/games?title={query}&limit=1"
    try:
        response = requests.get(url)
        data = response.json()
        
        if not data:
            return {"decision": "error", "message": f"Game '{query}' not found."}
            
        # Extract the official ID and Title
        game_id = data[0]['gameID']
        official_title = data[0]['external']
        
        return {"game_id": game_id, "game_title": official_title}
        
    except Exception as e:
        return {"decision": "error", "message": f"API Error: {str(e)}"}



In [4]:
# Node 2 - Fetch Price

def fetch_prices_node(state: DealState):
    """
    Step 2: Uses the ID from Step 1 to fetch live pricing from 25+ stores.
    """
    if state.get("decision") == "error":
        return {} # Skip if previous step failed
        
    game_id = state['game_id']
    print(f"--- 2. FETCHING: Getting live prices for ID {game_id}... ---")
    
    # REAL DATA SOURCE: Fetching specific deal details
    url = f"https://www.cheapshark.com/api/1.0/games?id={game_id}"
    try:
        response = requests.get(url)
        data = response.json()
        
        # 'deals' is a list of prices from Steam, Epic, etc.
        all_deals = data.get('deals', [])
        return {"deals": all_deals}
        
    except Exception as e:
        return {"decision": "error", "message": f"Price Fetch Error: {str(e)}"}
    
    

In [5]:
# Node 3 - analyzer

def analyzer_node(state: DealState):
    """
    Step 3: Busines Logic. Find the cheapest price and compares to target. 
    """
    if state.get("decision") == "error":
        return {} # Skip if previous step failed
    
    deals = state["deals"]
    target = state["target_price"]

    if not deals:
        return {"decision": "wait", "message": "No deals found"}

    # Logic: Sort by price (ascending) to fid the absolute cheapest price. 
    # Real world data comes as strings, need to cast to float

    sorted_deals = sorted(deals, key=lambda x: float(x['price']))
    cheapest_deal = sorted_deals[0]

    current_price = float(cheapest_deal['price'])
    store_id = cheapest_deal['storeID']

    print(f"--- 3. ANALYZING: Found dest deal at ${current_price} at store {store_id}... ---")
   
    # Make a decision based on the price
    if current_price <= target:
       return {
        "decision": "buy", 
        "best_deal": cheapest_deal,
        "message": f"Found deal at ${current_price} at store {store_id}"
        }
    else:
       return {
        "decision": "wait", 
        "best_deal": cheapest_deal,
        "message": f"Wait for a better deal. All current deals are too expensive. Best is ${current_price} at store {store_id}, but you want ${target}"
        }

In [6]:
# Node 4 - Notifier 

def notifier_node(state: DealState):
    """
    Step 4: Notify the user if the price is below the target price
    """
    color = "\033[92m" if state['decision'] == 'buy' else "\033[93m" # Green or Yellow
    reset = "\033[0m"
    
    print(f"\n{color}>>> NOTIFICATION: {state['message']} <<<{reset}\n")
    return {}

In [7]:
# Graph Construction

workflow = StateGraph(DealState)

# Add Nodes
workflow.add_node("search", search_game_node)
workflow.add_node("fetch", fetch_prices_node)
workflow.add_node("analyze", analyzer_node)
workflow.add_node("notify", notifier_node)

In [8]:
# Add edges
workflow.set_entry_point("search")

# Conditional Routing from search
# If search fails, go straight to notify. If valid, go to fetch.
def check_search_success(state):
    if state.get("decision") == "error":
        return "notify"
    return "fetch"

workflow.add_conditional_edges(
    "search",
    check_search_success,
    {
        "notify": "notify",
        "fetch": "fetch"
    }
)

workflow.add_edge("fetch", "analyze")
workflow.add_edge("analyze", "notify")
workflow.add_edge("notify", END)

In [9]:
# Compile

app = workflow.compile()

In [10]:
# Real life simulation

# We want "Batman" for under $5.
print("--- RUN 1. Searching for a cheap Batman game...")

app.invoke({"search_query": "Batman Arkham Knight", "target_price": 5.00, "decision": "pending"})

--- RUN 1. Searching for a cheap Batman game...
--- 1. SEARCHING: Looking for 'Batman Arkham Knight' in public database... ---
--- 2. FETCHING: Getting live prices for ID 107598... ---
--- 3. ANALYZING: Found dest deal at $3.99 at store 7... ---

>>> NOTIFICATION: Found deal at $3.99 at store 7 <<<



{'search_query': 'Batman Arkham Knight',
 'target_price': 5.0,
 'game_id': '107598',
 'game_title': 'Batman: Arkham Knight',
 'deals': [{'storeID': '7',
   'dealID': 'Fdh6o7UglWCqL%2BkWBLp5CJAMuIjqtBC73u5pgfurngs%3D',
   'price': '3.99',
   'retailPrice': '19.99',
   'savings': '80.040020'},
  {'storeID': '15',
   'dealID': 'PoyZQqPow3a4usGP%2Bsak3nhwFov5qquV%2BauZgNoVWsY%3D',
   'price': '15.95',
   'retailPrice': '19.99',
   'savings': '20.210105'},
  {'storeID': '21',
   'dealID': '0c%2BFYcqCbhVyzPhi8M67OgWKh%2BLTF79W1PSnMYcEFqg%3D',
   'price': '15.99',
   'retailPrice': '19.99',
   'savings': '20.010005'},
  {'storeID': '29',
   'dealID': 'EMOtGCEkEL2Eyepoc4znHjUvu5w8NqmkU0TETIt7GOA%3D',
   'price': '15.99',
   'retailPrice': '19.99',
   'savings': '20.010005'},
  {'storeID': '23',
   'dealID': '6ooKBSs5XsqUPxaLR4k9S1kG7w7Icd%2F8Ig0%2BliKLfmU%3D',
   'price': '16.57',
   'retailPrice': '19.99',
   'savings': '17.108554'},
  {'storeID': '27',
   'dealID': 'xrR1oyeiglETA0Mx6jb63CC%2

In [11]:
# Scenario 2: Unrealistic expectations
# We want a new AAA game for pennies.
print("\n--- RUN 2: Lowballing a premium game ---")
app.invoke({
    "search_query": "Elden Ring", 
    "target_price": 10.00,
    "decision": "pending"
})


--- RUN 2: Lowballing a premium game ---
--- 1. SEARCHING: Looking for 'Elden Ring' in public database... ---
--- 2. FETCHING: Getting live prices for ID 236717... ---
--- 3. ANALYZING: Found dest deal at $49.77 at store 23... ---

>>> NOTIFICATION: Wait for a better deal. All current deals are too expensive. Best is $49.77 at store 23, but you want $10.0 <<<



{'search_query': 'Elden Ring',
 'target_price': 10.0,
 'game_id': '236717',
 'game_title': 'ELDEN RING',
 'deals': [{'storeID': '23',
   'dealID': '0yH%2B27fnkwwPMEtLKiv52IGhamBesRW%2F9Ictx3%2B%2B2t0%3D',
   'price': '49.77',
   'retailPrice': '59.99',
   'savings': '17.036173'},
  {'storeID': '29',
   'dealID': 'Gp4puq5gha%2BLY%2F3XOqhecAhkCr9rYlZrIbA0mG%2Ff3Y4%3D',
   'price': '49.79',
   'retailPrice': '59.99',
   'savings': '17.002834'},
  {'storeID': '15',
   'dealID': '7uWDeqtvJFuw%2BJxO3176Lf3NbqvIahsS6PplDm%2BPS0Q%3D',
   'price': '53.88',
   'retailPrice': '59.99',
   'savings': '10.185031'},
  {'storeID': '27',
   'dealID': 'hYJjk5KtJE6yKkqeO7KhjbU4we6%2BjKYdUg9ciYIXn%2B8%3D',
   'price': '53.99',
   'retailPrice': '59.99',
   'savings': '10.001667'},
  {'storeID': '21',
   'dealID': 'QUMPDvMe9rGak%2B%2BSGXNun%2BRpZULEBeebaRQQVxoPAc8%3D',
   'price': '54.79',
   'retailPrice': '59.99',
   'savings': '8.668111'},
  {'storeID': '3',
   'dealID': 'WFUoyguZjL1LcvqOO70x5PA73uplVoZ